In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# KNN 모델을 적용한 재배환경 별 작물 종류 예측
참고자료: 나의 첫 머신러닝/딥러닝


![](http://wikibook.co.kr/images/cover/l/9791158392185.jpg)

In [2]:
df_sample =  pd.read_csv('../input/2022-ml-w4p1/sample.csv')
df_train = pd.read_csv('../input/2022-ml-w4p1/train.csv')
df_test = pd.read_csv('../input/2022-ml-w4p1/test.csv')
df_train.describe()

# 1.박스플롯을 통해 데이터 탐색(EDA)를 진행하였습니다.

In [3]:
df_train[['N','P','K','label']].groupby('label').boxplot(fontsize=20, figsize=(30,30))

In [4]:
df_train[['temperature','humidity','label']].groupby('label').boxplot(fontsize=20, figsize=(30,30))

In [5]:
df_train[['ph','label']].groupby('label').boxplot(fontsize=20, figsize=(30,30))

# 2. 모델의 정확도를 측정하기 위해 train data의 데이터를 8(훈련데이터):2(테스트데이터)비율로 나누어 줍니다.

In [6]:
#사이킷런의 train_test_split을 사용하면 코드 한 줄로 손쉽게 데이터를 나누어줄 수 있습니다.
from sklearn.model_selection import train_test_split
#df_train의 20%를 테스트 데이터로 분류해줍니다.
train, test = train_test_split(df_train, test_size=0.2)
print("학습데이터의 개수: " + str(train.shape[0]))
print("훈련용 데이터의 개수: " + str(test.shape[0]))


# 3. KNN 모델에 사용할 최적의 k(이웃의 개수)값을 찾습니다.

In [7]:
from sklearn.neighbors import KNeighborsClassifier #KNN라이브러리 추가
from sklearn.model_selection import cross_val_score

#교차검증을 수행할 K의 범위는 학습데이터의 절반으로 해주었습니다.
max_k_range = df_train.shape[0] // 2
k_list = []
for i in range(3, max_k_range, 2):
    k_list.append(i)

cross_validation_scores = []
x_train = train[['N','P','K','temperature', 'humidity', 'ph', 'rainfall']]
y_train = train[['label']]

# 교차 검증을 각 K를 대상으로 수행해 검증 결과를 저장해 줍니다.
for k in k_list:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, x_train, y_train.values.ravel(), cv=10, scoring='accuracy')
    cross_validation_scores.append(scores.mean())

In [8]:
#k값에 따른 정확도를 시각화해주었습니다.
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.plot(k_list, cross_validation_scores)
plt.xlabel('the number of k')
plt.ylabel('Accuracy')
plt.show()

In [9]:
k = k_list[cross_validation_scores.index(max(cross_validation_scores))]
print("최적의 k는:" + str(k) + "입니다.")

# 4. 훈련데이터로 모델을 학습시키고 정확도가 얼마나 되는지 확인해 줍니다.

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=k)

#예측에 사용되는 원료들 ㅎㅎ
x_train = train[['N','P','K','temperature', 'humidity', 'ph', 'rainfall']]
#KNN모델이 예측해야하는 label 칼럼에 있는 작물들
y_train = train[['label']]

#KNN 모델 학습
knn.fit(x_train, y_train.values.ravel())

#테스트 데이터에서 분류를 위해 사용될 속성들
x_test = test[['N','P','K','temperature', 'humidity', 'ph', 'rainfall']]
#테스트 데이터에 쓰일 정답 칼럼
y_test = test[['label']]

#정확도 예측 시작
pred = knn.predict(x_test)

#모델예측 정확도 출력
print("accuracy : " + str(accuracy_score(y_test.values.ravel(), pred)))

In [11]:
#정확도가 얼마나 되는지 dataframe을 통해 확인 해줍니다.
comparison = pd.DataFrame({'predicion':pred, 'ground_truth':y_test.values.ravel()})
comparison

In [17]:
df_train

# 5. 정답으로 제출해야할 실제 테스트 데이터에 학습된 모델을 적용시켜 줍니다.

In [19]:
#과제에서 지정해준 훈련용 데이터셋으로 훈련해줍니다.
x_train = df_train[['N','P','K','temperature', 'humidity', 'ph', 'rainfall']]
#KNN모델이 예측해야하는 label 칼럼에 있는 작물들
y_train = df_train[['label']]


In [22]:
df_test

In [29]:
#KNN 모델 학습
knn.fit(x_train, y_train.values.ravel())

#과제에서 지정해준 테스트 데이터에서 분류를 위해 사용될 속성들
x_test = df_test[['N','P','K','temperature', 'humidity', 'ph', 'rainfall']]

#정확도 예측 시작
pred = knn.predict(x_test)

#예측한 값
x_test = pd.concat([x_test, pd.DataFrame(pred)], axis=1)
x_test

In [32]:
x_test = x_test.rename(columns = {0:'추천작물'}, inplace=False)
x_test.to_csv('농작물추천.csv')